In [87]:
import pandas as pd


In [88]:
df=pd.read_csv('books.csv')


In [89]:
df

,Title,Author,Genre,Height,Publisher
0,Fundamentals of Wavelets,"Goswami, Jaideva",signal_processing,228,Wiley
1,Data Smart,"Foreman, John",data_science,235,Wiley
2,God Created the Integers,"Hawking, Stephen",mathematics,197,Penguin
3,Superfreakonomics,"Dubner, Stephen",economics,179,HarperCollins
4,Orientalism,"Said, Edward",history,197,Penguin
...,...,...,...,...,...
206,Structure and Randomness,"Tao, Terence",mathematics,252,NaN
207,Image Processing with MATLAB,"Eddins, Steve",signal_processing,241,NaN
208,Animal Farm,"Orwell, George",fiction,180,NaN
209,"Idiot, The","Dostoevsky, Fyodor",fiction,197,NaN


In [90]:
df.isna().sum()

Title         0
Author       24
Genre         0
Height        0
Publisher    96
dtype: int64

In [91]:
df.dropna(axis=0,inplace=True)

In [92]:
df.isna().sum()

Title        0
Author       0
Genre        0
Height       0
Publisher    0
dtype: int64

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [94]:
df['combined_features'] = df['Title'] + ' ' + df['Author'] + ' ' + df['Genre'] + ' ' + df['Publisher'].fillna('') + ' ' + df['Height'].astype(str)


In [95]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['combined_features'])


In [96]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = df.index[df['Title'] == title][0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    book_indices = [i[0] for i in sim_scores]
    return df['Title'].iloc[book_indices]

# Example usage
print(get_recommendations('Data Smart'))


0                      Fundamentals of Wavelets
2                      God Created the Integers
3                             Superfreakonomics
4                                   Orientalism
5    Nature of Statistical Learning Theory, The
Name: Title, dtype: object


In [97]:
import pickle

# Save the vectorizer and similarity matrix
with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

with open('cosine_sim.pkl', 'wb') as file:
    pickle.dump(cosine_sim, file)

# Load the vectorizer and similarity matrix
with open('vectorizer.pkl', 'rb') as file:
    loaded_vectorizer = pickle.load(file)

with open('cosine_sim.pkl', 'rb') as file:
    loaded_cosine_sim = pickle.load(file)

# Verify the loaded models
print(get_recommendations('Data Smart', cosine_sim=loaded_cosine_sim))


0                      Fundamentals of Wavelets
2                      God Created the Integers
3                             Superfreakonomics
4                                   Orientalism
5    Nature of Statistical Learning Theory, The
Name: Title, dtype: object


In [98]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# Sample DataFrame
data = {
    'Title': [
        'Fundamentals of Wavelets', 'Data Smart', 'God Created the Integers',
        'Superfreakonomics', 'Orientalism', 'Structure and Randomness',
        'Image Processing with MATLAB', 'Animal Farm', 'Idiot, The', 'Christmas Carol, A'
    ],
    'Author': [
        'Goswami, Jaideva', 'Foreman, John', 'Hawking, Stephen',
        'Dubner, Stephen', 'Said, Edward', 'Tao, Terence',
        'Eddins, Steve', 'Orwell, George', 'Dostoevsky, Fyodor', 'Dickens, Charles'
    ],
    'Genre': [
        'signal_processing', 'data_science', 'mathematics',
        'economics', 'history', 'mathematics',
        'signal_processing', 'fiction', 'fiction', 'fiction'
    ],
    'Height': [
        228, 235, 197, 179, 197, 252, 241, 180, 197, 196
    ],
    'Publisher': [
        'Wiley', 'Wiley', 'Penguin', 'HarperCollins', 'Penguin', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN'
    ]
}

df = pd.DataFrame(data)

# Combine features
df['combined_features'] = df['Title'] + ' ' + df['Author'] + ' ' + df['Genre'] + ' ' + df['Publisher'].fillna('') + ' ' + df['Height'].astype(str)

# Vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['combined_features'])

# Similarity calculation
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Recommendation function
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = df.index[df['Title'] == title][0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    book_indices = [i[0] for i in sim_scores]
    return df['Title'].iloc[book_indices]

# Save the vectorizer and similarity matrix
with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

with open('cosine_sim.pkl', 'wb') as file:
    pickle.dump(cosine_sim, file)

# Load the vectorizer and similarity matrix
with open('vectorizer.pkl', 'rb') as file:
    loaded_vectorizer = pickle.load(file)

with open('cosine_sim.pkl', 'rb') as file:
    loaded_cosine_sim = pickle.load(file)

# Verify the loaded models
print(get_recommendations('Data Smart', cosine_sim=loaded_cosine_sim))


0    Fundamentals of Wavelets
2    God Created the Integers
3           Superfreakonomics
4                 Orientalism
5    Structure and Randomness
Name: Title, dtype: object
